# Modeling 

# been there done that 

![been there](../Images/beenthere.jpg)

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn import metrics
from sklearn.pipeline import Pipeline

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


import torch 
from torch import cuda
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm 

from keras.models import save_model

2023-06-10 02:34:21.513478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
combined = pd.read_csv('../Data/combined.csv')

In [3]:
combined.head()

,artist_names,full_title,id,title,title_with_featured,url,featured_artists,artist,lyrics,cleaned_lyrics,clean_cleaned_lyrics
0,Pitbull (Ft. Vein (Producer)),11:59 by Pitbull (Ft. Vein (Producer)),654086,11:59,11:59 (Ft. Vein (Producer)),https://genius.com/Pitbull-11-59-lyrics,"[{'api_path': '/artists/7076', 'header_image_u...",1,11 Contributors11:59 Lyrics[Intro: Pitbull & V...,"\nWe shut down Times Square, extra extra read ...","['', 'pitbull', 'vein', 'vein', 'someone', 'po..."
1,Pitbull (Ft. Lil Jon),305 Anthem by Pitbull (Ft. Lil Jon),37215,305 Anthem,305 Anthem (Ft. Lil Jon),https://genius.com/Pitbull-305-anthem-lyrics,"[{'api_path': '/artists/107', 'header_image_ur...",1,17 Contributors305 Anthem Lyrics[Intro: Pitbul...,"\nMan I've been on the grind\n1 in the head, 1...","['', 'anthem', 'pitbull', 'lil', 'chico', 'pit..."
2,Pitbull (Ft. Jamie Drastik),305 In Paris by Pitbull (Ft. Jamie Drastik),9071119,305 In Paris,305 In Paris (Ft. Jamie Drastik),https://genius.com/Pitbull-305-in-paris-lyrics,"[{'api_path': '/artists/26923', 'header_image_...",1,1 Contributor305 In Paris Lyrics[Intro]\nWe're...,[Chorus: Pitbull]\nFlow so hard that chico get...,"['', 'paris', 'gonna', 'skate', 'one', 'song',..."
3,Pitbull,305 Til I Die by Pitbull,8988638,305 Til I Die,305 Til I Die,https://genius.com/Pitbull-305-til-i-die-lyrics,[],1,1 Contributor305 Til I Die Lyrics[Intro]\nIntr...,0,"['', 'til', 'die', 'mr', 'three', 'yeah', 'yea..."
4,Los Rafas (Ft. Pitbull),3bkzo RIMIX by Los Rafas (Ft. Pitbull),3972879,3bkzo RIMIX,3bkzo RIMIX (Ft. Pitbull),https://genius.com/Los-rafas-3bkzo-rimix-lyrics,"[{'api_path': '/artists/628', 'header_image_ur...",1,1 Contributor3bkzo RIMIX Lyrics[INTRO]\nMateín...,0,"['', 'rimix', 'mateín', 'checho', 'representan..."


In [4]:
combined.shape 

(1258, 11)

## Train-Test-Split

dale

In [5]:
X = combined['clean_cleaned_lyrics']
y = combined['artist']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

## Baseline

In [7]:
baseline_accuracy = combined['artist'].value_counts(normalize=True).max()
baseline_accuracy

0.5333863275039745

The baseline model predicts Pitbull more because he has more songs, and is accurate around 53% of the time 

## Logistic Regression 

In [8]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=['english', 'spanish'])),
    ('lr', LogisticRegression(random_state=42))
])

# Fit pipeline
pipeline.fit(X_train, y_train);

In [9]:
cross_val_score(pipeline, X_train, y_train, cv=3).mean()

0.8227500328914273

The cross validation score is fairly high (0.82), meaning it should do well on new data. The model so far is able to predict whether or not it's Pitbull of 82% of the songs.

In [10]:
print(f'Train: {pipeline.score(X_train, y_train)}')
print(f'Test: {pipeline.score(X_test, y_test)}')

Train: 0.9784090909090909
Test: 0.8809523809523809


The model has a high accuracy score on both the training set (97.8%) and the test set (88%), which shows that the model is performing well on both datasets but seems to be overfitting some on the training data.

I chose to not continue with the Logistic Regression model to spend more time with RNN and BERT, as I believed those models would be more fitting for this project.

## RNN 

*My computer gets very loud running these :)*

In [11]:
# Set random seed 
tf.random.set_seed(10)

# Prepare the data
lyrics = combined['clean_cleaned_lyrics'].values
labels = combined['artist'].values

# Tokenize lyrics 
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lyrics)
sequences = tokenizer.texts_to_sequences(lyrics)

vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(seq) for seq in sequences)

# Pad sequences to have the same length
padded_sequences = sequence.pad_sequences(sequences, maxlen=max_len)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, labels, test_size=0.2, random_state=42)

# Build the RNN model
embedding_dim = 100
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(LSTM(100))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32, callbacks=[early_stopping])

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')


Epoch 1/10


2023-06-10 02:34:31.975657: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 02:34:31.977528: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 02:34:31.979049: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - ETA: 0s - loss: 0.6902 - accuracy: 0.5447

2023-06-10 02:35:10.105867: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-06-10 02:35:10.107542: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-06-10 02:35:10.108743: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

32/32 [==============================] - 40s 1s/step - loss: 0.6902 - accuracy: 0.5447 - val_loss: 0.6769 - val_accuracy: 0.5714
Epoch 2/10
32/32 [==============================] - 37s 1s/step - loss: 0.6235 - accuracy: 0.7376 - val_loss: 0.5871 - val_accuracy: 0.7659
Epoch 3/10
32/32 [==============================] - 37s 1s/step - loss: 0.2442 - accuracy: 0.9334 - val_loss: 0.5156 - val_accuracy: 0.7976
Epoch 4/10
32/32 [==============================] - 42s 1s/step - loss: 0.0863 - accuracy: 0.9781 - val_loss: 0.5968 - val_accuracy: 0.7778
Epoch 5/10
32/32 [==============================] - 35s 1s/step - loss: 0.0644 - accuracy: 0.9841 - val_loss: 0.6625 - val_accuracy: 0.7302
Epoch 6/10
8/8 [==============================] - 2s 246ms/step - loss: 0.5156 - accuracy: 0.7976
Test Loss: 0.5156355500221252
Test Accuracy: 0.7976190447807312


The RNN varies in accuracy, between 70% to over 90% accuracy. 

In [12]:
# Save model to try in app
model.save('../RNN/model.h5')

## BERT

![why](../Images/whyrunning.jpg)

In [13]:
#https://discuss.huggingface.co/t/not-using-gpu-although-it-is-specified/14746/2
# Set the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# Split the data into train and test sets
train_text, test_text, train_labels, test_labels = train_test_split(combined['clean_cleaned_lyrics'], combined['artist'], test_size=0.2, random_state=42)

# Tokenize the text
max_length = 128 
train_tokens = tokenizer.batch_encode_plus(
    train_text.tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='pt'
)
test_tokens = tokenizer.batch_encode_plus(
    test_text.tolist(),
    truncation=True,
    padding=True,
    max_length=max_length,
    return_tensors='pt'
)

# Create TensorDatasets
train_dataset = TensorDataset(train_tokens['input_ids'], train_tokens['attention_mask'], torch.tensor(train_labels.tolist()))
test_dataset = TensorDataset(test_tokens['input_ids'], test_tokens['attention_mask'], torch.tensor(test_labels.tolist()))

# Create DataLoaders
batch_size = 16  
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, sampler=SequentialSampler(test_dataset))

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2) 
model.to(device)

# Set the optimizer and learning rate
optimizer = AdamW(model.parameters(), lr=2e-5)

# Train the model
num_epochs = 3  
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batches'):
        batch = tuple(t.to(device) for t in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    average_loss = total_loss / len(train_dataloader)
    print(f'Average training loss: {average_loss}')

# Evaluate the model
model.eval()
predictions = []
true_labels = []
for batch in tqdm(test_dataloader, desc='Evaluating', unit='batches'):
    batch = tuple(t.to(device) for t in batch)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predictions.extend(torch.argmax(logits, dim=1).tolist())
    true_labels.extend(inputs['labels'].tolist())

# Calculate accuracy
correct_predictions = np.array(predictions) == np.array(true_labels)
accuracy = np.sum(correct_predictions) / len(correct_predictions)
print(f'Test accuracy: {accuracy}')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Average training loss: 0.5042576262402156


Epoch 2/3: 100%|█████████████████████████████| 63/63 [18:13<00:00, 17.36s/batches]


Average training loss: 0.3594796810121763


Epoch 3/3: 100%|█████████████████████████████| 63/63 [20:03<00:00, 19.11s/batches]


Average training loss: 0.26615404964439454


Evaluating: 100%|████████████████████████████| 16/16 [01:15<00:00,  4.70s/batches]

Test accuracy: 0.9166666666666666


#### Save model for Streamlit app

I chose to use this model as it's results have been more contistently high (over 91%) and the way the BERT model works seems better fit for analyzing the generated lyrics, as it takes into account the relationships between words on either sides of a word. The model works significantly better than baseline (53%) and has worked well in the streamlit app. 

Also, I have spent too much time loading this thing to not use it. 

In [14]:
model.save_pretrained('../BERT/bert_model')

In [15]:
tokenizer.save_pretrained('../BERT/bert_tokenizer')

('../BERT/bert_tokenizer/tokenizer_config.json',
 '../BERT/bert_tokenizer/special_tokens_map.json',
 '../BERT/bert_tokenizer/vocab.txt',
 '../BERT/bert_tokenizer/added_tokens.json')

![count](../Images/count.JPG)

I have been listening to Pitbull for hours on end while doing this project. For research.  